In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [13]:
import sys,os
sys.path.append(os.pardir)
from common.functions import*
from common.gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size,weight_init_std=0.01):
        #가중치 초기화
        self.params={}
        self.params['W1']=weight_init_std*np.random.randn(input_size,hidden_size) #size(input,hidden)
        self.params['b1']=np.zeros(hidden_size)
        self.params['W2']=weight_init_std*np.random.randn(hidden_size,output_size)
        self.params['b2']=np.zeros(output_size)
    def predict(self,x):
        W1,W2=self.params['W1'],self.params['W2']
        b1,b2=self.params['b1'],self.params['b2']
        a1=np.dot(x,W1)+b1
        z1=sigmoid(a1)
        a2=np.dot(z1,W2)+b2
        y=softmax(a2)
        return y
    def loss(self,x,t):
        y=self.predict(x)
        
        return cross_entropy_error(y,t)
    def accuracy(self,x,t):
        y=self.predict(x)
        y=np.argmax(y,axis=1)
        t=np.argmax(t,axis=1)
        accuracy=np.sum(y==t)/float(x.shape[0])
        return accuracy
    def numerical_gradient(self,x,t):
        loss_W=lambda W:self.loss(x,t)
        grads={}
        grads['W1']=numerical_gradient(loss_W,self.params['W1'])
        grads['b1']=numerical_gradient(loss_W,self.params['b1'])
        grads['W2']=numerical_gradient(loss_W,self.params['W2'])
        grads['b2']=numerical_gradient(loss_W,self.params['b2'])
        
        return grads
        
    

In [14]:
net=TwoLayerNet(input_size=784,hidden_size=100,output_size=10)
net.params['W1'].shape
net.params['b1'].shape
net.params['W2'].shape
net.params['b2'].shape

(784, 100)

(100,)

(100, 10)

(10,)

In [15]:
x=np.random.rand(100,784) #100장 더미데이터
t=np.random.rand(100,10)
grads=net.numerical_gradient(x,t) #기울기 계산
grads['W1'].shape
grads['b1'].shape
grads['W2'].shape
grads['b2'].shape

(784, 100)

(100,)

(100, 10)

(10,)

In [ ]:
from dataset.mnist import load_mnist

(x_train,t_train),(x_train,t_test)=\
load_mnist(normalize=True,one_hot_label=True)
train_loss_list=[]

#하이퍼파라미터
iters_num=10000
train_size=x_train.shape[0]
batch_size=100 #미니배치 크기
learning_rate=0.1
network=TwoLayerNet(input_size=784,hidden_size=50,output_size=10)
for i in range(iters_num):
    #미니배치
    batch_mask=np.random.choice(train_size,batch_size)
    x_batch=x_train[batch_mask]
    t_batch=t_train[batch_mask]
    
    #기울기
    grad=network.numerical_gradient(x_batch,t_batch)
    for key in ('W1','b1','W2','b2'):
        network.params[key]-=learning_rate*grad[key]
    
    loss=network.loss(x_batch,t_batch)
    train_loss_list.append(loss)